# Representações Latentes e Autoencoders

Neste notebook, exploraremos o conceito de representações latentes e como os autoencoders podem ser utilizados para aprendê-las de forma não supervisionada. Abordaremos os seguintes tópicos:

* **Representações Latentes**:
    * Definição formal e o objetivo de aprender representações de dados em espaços de menor dimensionalidade.
    * Visualização do espaço latente com o algoritmo t-SNE (t-Distributed Stochastic Neighbor Embedding).
    * Exemplo prático de treinamento de um codificador (encoder) para a tarefa de classificação e a subsequente visualização de suas representações latentes para o dataset MNIST.

* **Autoencoders**:
    * Arquitetura fundamental de um autoencoder (encoder-decoder).
    * Implementação e treinamento de um autoencoder para reconstruir imagens do MNIST.
    * Visualização de imagens originais e reconstruídas.

* **Denoising Autoencoders**:
    * Conceito e formulação de autoencoders com remoção de ruído.
    * Aplicação prática na remoção de ruído de imagens do MNIST.

## Representações Latentes

No contexto de Deep Learning, uma representação latente é uma codificação interna dos dados de entrada que emerge como resultado do processo de treinamento de uma rede neural para uma tarefa específica. Em vez de ser um resultado de um algoritmo de redução de dimensionalidade pré-definido, o espaço latente é aprendido dinamicamente.

As redes neurais profundas são compostas por uma sequência de camadas. Cada camada executa uma transformação sobre seu dado de entrada, e o resultado é uma representação progressivamente mais abstrata. A saída de qualquer camada intermediária de uma rede pode ser considerada uma representação latente.

A estrutura e as propriedades deste espaço latente são diretamente influenciadas pela função objetivo (loss function) que o modelo otimiza.

Em Aprendizagem Supervisionada (e.g., Classificação), o modelo é treinado para minimizar um erro de classificação (como a Entropia Cruzada). Para isso, o algoritmo de backpropagation ajusta os pesos da rede de forma a transformar os dados de entrada em representações internas que tornem as classes o mais separáveis possível. Um espaço latente ideal, neste caso, agrupará amostras da mesma classe em regiões coesas e distintas, idealmente permitindo uma separação linear por parte das camadas finais da rede. A representação aprende a reter apenas as características discriminativas para a tarefa.

Formalmente, o mapeamento para o espaço latente é uma função parametrizada $\mathbf{z} = f_{\theta}(x)$, onde $\theta$ representa os pesos da rede (o *encoder*) e $\mathbf{z} \in \mathbb{R}^m$ é o vetor latente. Esses parâmetros $\theta$ são aprendidos através da otimização de gradiente descendente para minimizar a função de perda da tarefa final.

In [ ]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from torchvision import datasets
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms

# Transformação
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.1307,), (0.3081,))
])

# Datasets
train_data = datasets.MNIST("./data", train=True, download=True, transform=transform)
test_data  = datasets.MNIST("./data", train=False, download=True, transform=transform)

# Subsets menores
train_subset = Subset(train_data, torch.randperm(len(train_data))[:10000])
val_subset   = Subset(test_data,  torch.randperm(len(test_data))[:1000])

# DataLoaders
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_subset, batch_size=64)

print(len(train_subset), len(val_subset))

### Treinando um Modelo para Aprender Representações

Para gerar um espaço latente significativo, treinaremos uma Rede Neural na tarefa de classificação do MNIST. As camadas intermediárias desta rede aprenderão a extrair características relevantes dos dados, e a saída de uma dessas camadas servirá como nossa representação latente.

In [ ]:
class Classifier(nn.Module):
    def __init__(self, latent_dim=64):
        super().__init__()
        self.flatten = nn.Flatten()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(64, latent_dim),
            nn.ReLU(),
        )
        self.classifier_head = nn.Linear(latent_dim, 10)

    def encode(self, x):
        x = self.flatten(x)
        latent = self.encoder(x)
        return latent

    def forward(self, x):
        latent = self.encode(x)
        logits = self.classifier_head(latent)
        return logits

model = Classifier().to(device)
print(model)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}
epochs = 10

model = model.to(device)

for epoch in range(epochs):
    # ---- Treino ----
    model.train()
    total_loss, correct, total = 0, 0, 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(output, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
    
    history["train_loss"].append(total_loss / len(train_loader))
    history["train_acc"].append(100 * correct / total)

    # ---- Validação ----
    model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item()
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    
    history["val_loss"].append(val_loss / len(val_loader))
    history["val_acc"].append(100 * correct / total)

    print(f"Epoch {epoch+1}/{epochs} | "
          f"Train Loss: {history['train_loss'][-1]:.4f}, "
          f"Train Acc: {history['train_acc'][-1]:.2f}% | "
          f"Val Loss: {history['val_loss'][-1]:.4f}, "
          f"Val Acc: {history['val_acc'][-1]:.2f}%")

# ---- Plots ----
plt.figure(figsize=(12,5))

# Loss
plt.subplot(1,2,1)
plt.plot(history["train_loss"], label="Train Loss")
plt.plot(history["val_loss"], label="Val Loss", linestyle="--")
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

# Accuracy
plt.subplot(1,2,2)
plt.plot(history["train_acc"], label="Train Acc")
plt.plot(history["val_acc"], label="Val Acc", linestyle="--")
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.legend()

plt.tight_layout()
plt.show()

### Visualização com t-SNE

O t-Distributed Stochastic Neighbor Embedding (t-SNE) é um algoritmo não linear de redução de dimensionalidade utilizado primariamente para a visualização de datasets de alta dimensão. Sua principal função é mapear a similaridade entre pontos de dados de um espaço de alta dimensão para um espaço de baixa dimensão (geralmente 2D ou 3D).

O t-SNE modela a probabilidade condicional de que um ponto $x_i$ escolheria $x_j$ como seu vizinho, baseando-se em uma distribuição Gaussiana centrada em $x_i$. Em seguida, ele tenta construir uma distribuição de probabilidade similar sobre os pontos no mapa de baixa dimensão. A otimização é realizada minimizando a divergência de Kullback-Leibler (KL) entre as duas distribuições de probabilidades, a do espaço original e a do espaço de baixa dimensão. O resultado é um mapa que revela a estrutura de clusters dos dados.

In [ ]:
from sklearn.manifold import TSNE

model.eval()
all_latents = []
all_labels = []

with torch.no_grad():
    for X, y in val_loader:
        X = X.to(device)
        latent = model.encode(X)
        all_latents.append(latent.cpu().numpy())
        all_labels.append(y.cpu().numpy())

latent_space_test = np.concatenate(all_latents, axis=0)
labels_test = np.concatenate(all_labels, axis=0)

tsne = TSNE(n_components=2, random_state=42, verbose=1, perplexity=30, n_iter=1000)
tsne_results = tsne.fit_transform(latent_space_test)

plt.figure(figsize=(12, 10))
scatter = plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=labels_test, cmap='tab10', s=10)
plt.title('Visualização do Espaço Latente com t-SNE', fontsize=16)
plt.xlabel('Componente t-SNE 1')
plt.ylabel('Componente t-SNE 2')
plt.legend(handles=scatter.legend_elements()[0], labels=list(range(10)))
plt.grid(True)
plt.show()

## Autoencoders

Um Autoencoder é um tipo de rede neural artificial utilizada para aprender representações de dados de forma não supervisionada. A arquitetura é composta por duas partes principais:

1.  **Encoder ($f$)**: Mapeia a entrada $x$ para uma representação latente $z$ de menor dimensão. $z = f(x)$.
2.  **Decoder ($g$)**: Tenta reconstruir a entrada original a partir da representação latente $z$. $\hat{x} = g(z)$.

O objetivo do treinamento é minimizar o erro de reconstrução, que é a diferença entre a entrada original $x$ e a sua reconstrução $\hat{x}$. Uma função de perda comum para essa tarefa é o Erro Quadrático Médio (Mean Squared Error - MSE).

$$ \mathcal{L}(x, \hat{x}) = \mathcal{L}(x, g(f(x))) = \frac{1}{N} \sum_{i=1}^{N} (x_i - \hat{x}_i)^2 $$

O gargalo informacional imposto pela camada latente força o autoencoder a aprender apenas as variações mais importantes nos dados.

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, encoding_dim=8):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, encoding_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28 * 28),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

ae_model = Autoencoder().to(device)
print(ae_model)

In [ ]:
ae_criterion = nn.MSELoss()
ae_optimizer = torch.optim.Adam(ae_model.parameters(), lr=1e-3, weight_decay=1e-8)

In [ ]:
epochs = 20
ae_model.train()

train_losses = []

for epoch in range(epochs):
    epoch_loss = 0.0
    for X, _ in train_loader:
        X = X.view(X.size(0), -1).to(device)
        
        # Forward
        recon = ae_model(X)
        loss = ae_criterion(recon, X)

        # Backward
        ae_optimizer.zero_grad()
        loss.backward()
        ae_optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")


plt.figure(figsize=(8,5))
plt.plot(range(1, epochs+1), train_losses, marker="o")
plt.title("Training Loss (Autoencoder)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.show()

### Visualização das Reconstruções

Após o treinamento, podemos passar imagens do conjunto de teste pelo autoencoder para obter suas reconstruções. Comparar visualmente as imagens originais com as reconstruídas nos dá uma avaliação qualitativa do desempenho do modelo.

In [ ]:
# Visualização das imagens reconstruídas
ae_model.eval()
n = 10

with torch.no_grad():
    # Pega um batch de dados de validação
    data_iter = iter(val_loader)
    images, _ = next(data_iter)
    images_flat = images.view(images.size(0), -1).to(device)
    
    # Gera as reconstruções
    reconstructed_flat = ae_model(images_flat)
    
    # Converte para numpy para plotar
    original_images = images.cpu().numpy()
    reconstructed_images = reconstructed_flat.view(-1, 1, 28, 28).cpu().numpy()

In [ ]:
plt.figure(figsize=(20, 4))

for i in range(n):
    # Imagem original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(original_images[i].squeeze(), cmap='gray', vmin=0, vmax=1)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n//2:
        ax.set_title('Imagens Originais')

    # Imagem reconstruída
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(reconstructed_images[i].squeeze(), cmap='gray', vmin=0, vmax=1)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n//2:
        ax.set_title('Imagens Reconstruídas')

plt.show()

In [ ]:
ae_model.eval()
ae_all_latents = []
ae_all_labels = []

with torch.no_grad():
    for X, y in val_loader:
        X = X.view(X.size(0), -1).to(device)
        latent = ae_model.encoder(X)
        ae_all_latents.append(latent.cpu().numpy())
        ae_all_labels.append(y.cpu().numpy())

ae_latent_space_test = np.concatenate(ae_all_latents, axis=0)
ae_labels_test = np.concatenate(ae_all_labels, axis=0)

tsne = TSNE(n_components=2, random_state=42, verbose=1, perplexity=30, n_iter=1000)
tsne_results = tsne.fit_transform(ae_latent_space_test)

plt.figure(figsize=(12, 10))
scatter = plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=labels_test, cmap='tab10', s=10)
plt.title('Visualização do Espaço Latente com t-SNE', fontsize=16)
plt.xlabel('Componente t-SNE 1')
plt.ylabel('Componente t-SNE 2')
plt.legend(handles=scatter.legend_elements()[0], labels=list(range(10)))
plt.grid(True)
plt.show()

## Denoising Autoencoders

Um Denoising Autoencoder é uma variação do autoencoder padrão que é treinado para reconstruir uma entrada limpa a partir de uma versão corrompida (ruidosa) da mesma. A hipótese é que, para realizar essa tarefa, o modelo é forçado a aprender características mais robustas e a extrair a estrutura subjacente dos dados, em vez de simplesmente aprender uma função identidade.

O processo de treinamento envolve:
1.  Corromper a imagem de entrada $x$ para obter $\tilde{x}$.
2.  Alimentar $\tilde{x}$ ao encoder para obter $z = f(\tilde{x})$.
3.  Alimentar $z$ ao decoder para obter a reconstrução $\hat{x} = g(z)$.
4.  Minimizar a perda entre a reconstrução $\hat{x}$ e a imagem original *limpa* $x$.

$$ \mathcal{L}(x, \hat{x}) = \mathcal{L}(x, g(f(\tilde{x}))) $$

In [ ]:
noise_factor = 0.4

data_iter = iter(val_loader)
images, _ = next(data_iter)
images_clean_flat = images.view(images.size(0), -1).to(device)

# Cria a versão com ruído
noise = torch.randn_like(images_clean_flat) * noise_factor
images_noisy_flat = torch.clip(images_clean_flat + noise, 0., 1.)

# Prepara para plotagem
original_imgs = images_clean_flat.cpu().numpy().reshape(-1, 28, 28)
noisy_imgs = images_noisy_flat.cpu().numpy().reshape(-1, 28, 28)

plt.figure(figsize=(20, 6))

for i in range(n):
    # Imagem original (limpa)
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(original_imgs[i], cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n//2:
        ax.set_title('Original')

    # Imagem com ruído (entrada do modelo)
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(noisy_imgs[i], cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n//2:
        ax.set_title('Com Ruído')

In [ ]:
denoising_ae_model = Autoencoder().to(device)
dae_loss_fn = nn.MSELoss()
dae_optimizer = torch.optim.Adam(denoising_ae_model.parameters(), lr=1e-3, weight_decay=1e-8)

In [ ]:
import matplotlib.pyplot as plt

epochs = 25
denoising_ae_model.train()

train_losses = []

for epoch in range(epochs):
    epoch_loss = 0.0
    for X, _ in train_loader:
        X_clean = X.view(X.size(0), -1).to(device)

        # Adiciona ruído
        noise = torch.randn_like(X_clean) * noise_factor
        X_noisy = torch.clamp(X_clean + noise, 0., 1.)

        # Predição e perda (com imagem limpa)
        recon = denoising_ae_model(X_noisy)
        loss = dae_loss_fn(recon, X_clean)

        # Backpropagation
        dae_optimizer.zero_grad()
        loss.backward()
        dae_optimizer.step()

        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")


plt.figure(figsize=(8,5))
plt.plot(range(1, epochs+1), train_losses, marker="o")
plt.title("Training Loss (Denoising Autoencoder)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.show()

In [ ]:
denoising_ae_model.eval()
n = 10

with torch.no_grad():
    data_iter = iter(val_loader)
    images, _ = next(data_iter)
    images_clean_flat = images.view(images.size(0), -1).to(device)
    
    # Cria a versão com ruído
    noise = torch.randn_like(images_clean_flat) * noise_factor
    images_noisy_flat = torch.clip(images_clean_flat + noise, 0., 1.)
    
    # Gera a reconstrução (denoised)
    denoised_flat = denoising_ae_model(images_noisy_flat)
    
    # Prepara para plotagem
    original_imgs = images_clean_flat.cpu().numpy().reshape(-1, 28, 28)
    noisy_imgs = images_noisy_flat.cpu().numpy().reshape(-1, 28, 28)
    denoised_imgs = denoised_flat.cpu().numpy().reshape(-1, 28, 28)

In [ ]:
plt.figure(figsize=(20, 6))

for i in range(n):
    # Imagem original (limpa)
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(original_imgs[i], cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n//2:
        ax.set_title('Original')

    # Imagem com ruído (entrada do modelo)
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(noisy_imgs[i], cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n//2:
        ax.set_title('Com Ruído')

    # Imagem reconstruída (denoised)
    ax = plt.subplot(3, n, i + 1 + 2*n)
    plt.imshow(denoised_imgs[i], cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n//2:
        ax.set_title('Reconstruída (Denoised)')
plt.show()